In [1]:
from pop import Util, Pilot
from pop.Util import imshow
import numpy as np
import cv2
import math
import time
import threading
import sys
# import queue
from scipy.ndimage import rotate
import multiprocessing as mp
import re

bot = Pilot.SerBot()
Util.enable_imshow()

/usr/local/lib/python3.6/dist-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
global white
global azure
global aqua
global red
global black
global yellow
global green
global steelblue
global rainbow
global orange
steelblue = (180,130,70)
green = (0,255,108)
white = (255,255,255)
azure = (255,255,240)
aqua = (255,255,0)
red = (0,0,255)
black = (0,0,0)
yellow = (0,255,255)
orange = (0, 165, 255)
rainbow = [(255,255,255), (0,0,255), (0, 165, 255), (0,255,255), (0,255,108), (255,255,0)]

<h3> ICP Algorithm (Iterative Closest Point) <h3/>

In [3]:
def realize_map(baseIMGSIZE_x, baseIMGSIZE_y, MapSerbot_x, MapSerbot_y, q1, q2, q3, q4, q5):
    import math
    import subprocess as sp, time
    from IPython.display import display, Javascript
    from ipywidgets import widgets
    
    img_size_x = 400
    img_size_y = 400
    
    SerBotPoint_x = int(img_size_x//2)
    SerBotPoint_y = int(img_size_y//2)
    
    dtime = time.time()
    imgShowTime = 0
    
    # 현재 위치에서 주변의 장애물 탐지
    def changeLocation_x(before_x, before_y, after_x, after_y, count):
        movePoint = 5
        sumCount = count
        dis_tuple = closestPoint(before_x, before_y, after_x, after_y)
        avr = avr_distance(dis_tuple)

        ch_after_x = after_x + movePoint
        ch_dis_tuple = closestPoint(before_x, before_y, ch_after_x, after_y)
        ch_avr = avr_distance(ch_dis_tuple)

        if avr > ch_avr:
            sumCount = count + 1
            vch_after_x = ch_after_x + movePoint
            vch_dis_tuple = closestPoint(before_x, before_y, vch_after_x, after_y)
            vch_avr = avr_distance(vch_dis_tuple)

            if ch_avr <= vch_avr :
                return ch_after_x, sumCount

            final_x, end_count = changeLocation_x(before_x, before_y, ch_after_x, after_y, sumCount)
            return final_x, end_count 
        else :
            sumCount = count - 2
            ch_after_x = after_x - (movePoint * 2)
            final_x, end_count = changeLocation_x(before_x, before_y, ch_after_x, after_y, sumCount)
            return final_x, end_count

    def changeLocation_y(before_x, before_y, after_x, after_y, count):
        movePoint = 5
        sumCount = count
        dis_tuple = closestPoint(before_x, before_y, after_x, after_y)
        avr = avr_distance(dis_tuple)

        ch_after_y = after_y + movePoint
        ch_dis_tuple = closestPoint(before_x, before_y, after_x, ch_after_y)
        ch_avr = avr_distance(ch_dis_tuple)

        if avr > ch_avr:
            sumCount = count + 1
            vch_after_y = ch_after_y + movePoint
            vch_dis_tuple = closestPoint(before_x, before_y, after_x, vch_after_y)
            vch_avr = avr_distance(vch_dis_tuple)

            if ch_avr <= vch_avr :
                return ch_after_y, sumCount

            final_y, end_count = changeLocation_y(before_x, before_y, after_x, ch_after_y, sumCount)
            return final_y, end_count
        else :
            sumCount = count - 2
            ch_after_y = after_y - (movePoint * 2)
            final_y, end_count = changeLocation_y(before_x, before_y, after_x, ch_after_y, sumCount)
            return final_y, end_count
    
    def rotate_point(before_x, before_y, other_location):
        moveAngle = 10
        location_x, location_y = (before_x, before_y)
        other_x, other_y = np.where(other_location==1)

        r_dis_tuple = closestPoint(location_x, location_y, other_x, other_y)
        r_avr = avr_distance(r_dis_tuple)

        r_other_location = rotate(other_location, angle=moveAngle, reshape=False)
        r_other_x, r_other_y = np.where(r_other_location==1)
        rr_dis_tuple = closestPoint(location_x, location_y, r_other_x, r_other_y)
        rr_avr = avr_distance(rr_dis_tuple)

        if r_avr > rr_avr:
            vr_other_location = rotate(r_other_location, angle=moveAngle, reshape=False)
            vr_other_x, vr_other_y = np.where(vr_other_location==1)
            vrr_dis_tuple = closestPoint(r_other_x, r_other_y, vr_other_x, vr_other_y)
            vrr_avr = avr_distance(vrr_dis_tuple)

            if rr_avr <= vrr_avr:
                return r_other_x, r_other_y

            final_x, final_y = rotate_point(before_x, before_y, r_other_location)
            return final_x, final_y
        else :
            re_moveAngle = moveAngle * -2
            r_other_location = rotate(other_location, angle=re_moveAngle, reshape=False)
            final_x, final_y = rotate_point(before_x, before_y, r_other_location)
            return final_x, final_y
        
    ### before_location과 after_location이 있을때 각 점마다 제일 가까운 점을 찾고 점 사이의 거리를 구해야 한다.
    # dis_tuple : 제일 가까운 점의 거리와 좌표를 저장
    def closestPoint(before_x, before_y, after_x, after_y):
        dis_tuple = np.zeros((len(before_x), 2), dtype=np.uint16)
        for i in range(len(before_x)):
            dis_x = before_x[i] - after_x[0]
            dis_y = before_y[i] - after_y[0]
            dis = math.pow(dis_x, 2) + math.pow(dis_y, 2)
            dis_min = math.fabs(math.sqrt(dis))
            for j in range(len(after_x)):
                dis_x_j = before_x[i] - after_x[j]
                dis_y_j = before_y[i] - after_y[j]
                dis_j = math.pow(dis_x_j, 2) + math.pow(dis_y_j, 2)
                distance = math.fabs(math.sqrt(dis_j))
                if dis_min > distance:
                    dis_tuple[i][0] = distance
                    dis_tuple[i][1] = j
                    pass
        return dis_tuple

    # 각 점마다의 거리를 더한 후 평균을 구한다.
    def avr_distance(dis_tuple):
        sumdistance = 0
        for i in range(len(dis_tuple)):
            sumdistance += dis_tuple[i][0]
        avr = round(sumdistance / len(dis_tuple), 3)
        return avr

    def change_one(before_x, before_y, after_x, after_y, count_x, count_y):
        final_y, y_count = changeLocation_y(before_x, before_y, after_x, after_y, count_y)
        final_x, x_count = changeLocation_x(before_x, before_y, after_x, final_y, count_x)
        return final_x, final_y, x_count, y_count
    
    indexColor = 0
    def photo(final_x, final_y, MapSerbot_x, MapSerbot_y):
        nonlocal indexColor
        baseMapIMG = cv2.imread('BaseArray.jpg', cv2.IMREAD_COLOR)
        index_xp = index_yp = 0
        baseMapIMG = cv2.circle(baseMapIMG, (MapSerbot_x, MapSerbot_y), 13, tuple(rainbow[indexColor]), -1)
        for i in range(len(final_x)):
            index_xp = final_x[i] + MapSerbot_x - (img_size_x//2)
            index_yp = final_y[i] + MapSerbot_y - (img_size_y//2)
            baseMapIMG = cv2.circle(baseMapIMG, (index_yp, index_xp), 4, tuple(rainbow[indexColor]), -1)
            pass
        cv2.imwrite('BaseArray.jpg', baseMapIMG)
        baseMapIMG = cv2.imread('BaseArray.jpg', cv2.IMREAD_COLOR)
        indexColor += 1
        indexColor = indexColor % 6
        q1.put(baseMapIMG)

    filecount = 0
    while True:
        if (filecount % 10) >= 24:
            with open("/home/soda/Project/python/notebook/KJJ/project/point.txt", "r") as f:
                data = f.readlines()
                pass
        else :
            pass

        if filecount == 0:
            with open("/home/soda/Project/python/notebook/KJJ/project/point.txt", "r") as f:
                data = f.readlines()
                pass
            x_count = y_count = 0
            d_x = data[filecount]
            d_y = data[filecount+1]
            sd_x = re.findall("\d+", d_x)
            sd_y = re.findall("\d+", d_y)
            ind_x = list(map(int, sd_x))
            ind_y = list(map(int, sd_y))
            before_x = np.array(ind_x)
            before_y = np.array(ind_y)
            filecount = filecount + 2
            pass
        
        if filecount < len(data):
            d_x = data[filecount]
            d_y = data[filecount+1]
            sd_x = re.findall("\d+", d_x)
            sd_y = re.findall("\d+", d_y)
            ind_x = list(map(int, sd_x))
            ind_y = list(map(int, sd_y))
            change_x = np.array(ind_x)
            change_y = np.array(ind_y)
            filecount = filecount + 2
        else :
            break
        
        for i in range(2):
            change_x, change_y, x_count, y_count = change_one(before_x, before_y, change_x, change_y, x_count, y_count)
            final_location = np.zeros((img_size_x, img_size_y), dtype=np.uint16)
            for j in range(len(change_x)):
                if change_x[j] < img_size_x and change_y[j] < img_size_y:
                    final_location[change_x[j]][change_y[j]] = 1
            change_x, change_y = rotate_point(before_x, before_y, final_location)

        # MapSerbot_x, y을 기준으로 지도가 작성되어 감
        before_x = change_x
        before_y = change_y
        before_location = np.array(final_location, dtype=np.uint16)
        MapSerbot_x = MapSerbot_x + x_count * 5
        MapSerbot_y = MapSerbot_y + y_count * 5
        photo(change_x, change_y, MapSerbot_x, MapSerbot_y)

In [4]:
def subprocessor(q1, q2, q3, q4, q5):
    from pop import LiDAR
    
    lidar = LiDAR.Rplidar()
    lidar.connect()
    lidar.startMotor()
    
    img_size_x = 400
    img_size_y = 400
    
    SerBotPoint_x = int(img_size_x//2)
    SerBotPoint_y = int(img_size_y//2)
    
    steelblue = (180,130,70)
    green = (0,255,108)
    white = (255,255,255)
    azure = (255,255,240)
    aqua = (255,255,0)
    red = (0,0,255)
    black = (0,0,0)
    yellow = (0,255,255)
    orange = (0, 165, 255)
    rainbow = [(255,255,255), (0,0,255), (0, 165, 255), (0,255,255), (0,255,108), (255,255,0)]
    
    def location():
        location = np.zeros((img_size_x, img_size_y), dtype=np.uint16)
        vector = lidar.getVectors()
        distance = vector[:,1]
        distance = distance // 10
        degree = vector[:,0]
        for i in range(len(degree)):
            # 기준 지점 = SerBot의 현재 위치
            # 라인의 끝 지점(x좌표, y좌표) 지정
            degree[i] = round(degree[i], 2)
            searchPoint_x = SerBotPoint_x + int(abs(distance[i]*(math.cos(math.radians(degree[i])))))
            searchPoint_y = SerBotPoint_y + int(abs(distance[i]*(math.sin(math.radians(degree[i])))))
            if searchPoint_x < img_size_x and searchPoint_y < img_size_y:
                location[searchPoint_x][searchPoint_y] = 1        
        return location
    
    # 새로 측정한 2차원 배열
    def after_Location():
        after_location = location()
        after_x, after_y = np.where(after_location==1)
        return after_x, after_y

#     # 기준으로 삼을 2차원 배열
#     before_location = location()
#     before_x, before_y = np.where(before_location==1)
    
    while True:
        after_x, after_y = after_Location()
        with open("/home/soda/Project/python/notebook/KJJ/project/point.txt", "a") as f:
            f.write(str(after_x.tolist())+"\n")
            f.write(str(after_y.tolist())+"\n")

        time.sleep(0.2)
        # 몇 초 후에 파일 업데이트하는 부분 추가

In [5]:
def stick():
    bot.joystick()

In [ ]:
dtime = time.time()
imgShowTime = 0
if __name__ == '__main__':
#     with open("/home/soda/Project/python/notebook/KJJ/project/point.txt", "w") as f:
#         pass
    q1 = mp.Queue(1)
    q2 = mp.Queue(1)
    q3 = mp.Queue(1)
    q4 = mp.Queue(1)
    q5 = mp.Queue(1)
    
    baseIMGSIZE_x = 1500
    baseIMGSIZE_y = 1500
    MapSerbot_x, MapSerbot_y = int(baseIMGSIZE_x * 0.5) , int(baseIMGSIZE_y * 0.5)
    MapSerBot = [MapSerbot_x, MapSerbot_y]
    
    t1 = threading.Thread(target=stick)
#     t2 = threading.Thread(target=subprocessor, args=(q1, q2, q3, q4, q5))
#     t2 = mp.Process(target=subprocessor, args=(q1, q2, q3, q4, q5), daemon=True)
#     time.sleep(2)
#     t3 = mp.Process(target=realize_map, args=(baseIMGSIZE_x, baseIMGSIZE_y, MapSerbot_x, MapSerbot_y, q1, q2, q3, q4, q5), daemon=True)
    t3 = threading.Thread(target=realize_map, args=(baseIMGSIZE_x, baseIMGSIZE_y, MapSerbot_x, MapSerbot_y, q1, q2, q3, q4, q5))

#     t1.start()
#     t2.start()
    t3.start()
#     bot.joystick()

    baseMapPoint = np.zeros((baseIMGSIZE_x, baseIMGSIZE_y), dtype=np.uint16)
    cv2.imwrite('BaseArray.jpg', baseMapPoint)
    
    while True:
#         data = q2.get()
#         print(data)
        baseMapImg = q1.get()
        cv2.imshow('BaseArray.jpg', baseMapImg, width=500, height=500)

    t3.join()
    pass

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…